In this notebook you'll find a range of tests showcasing how leakage is detected using our repostory. 
This notebook is divided into two sections where each section is analyzing one type of data.

## Data
Two different images of type MEG (Magnetoencephalography) and EEG (Electroencephalography) has been analyzed in this notebook. Each time you run a test will be downloaded and deleted at the end. Read more about MEG data in this [link](https://openneuro.org/datasets/ds000117/versions/1.0.6) and EEG data in this [link](https://osf.io/thsqg/).


## Methods:
We have tested the leakage score using three known methods. 
- [[Pearson correlation]](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://en.wikipedia.org/wiki/Pearson_correlation_coefficient&ved=2ahUKEwic1fmek9KNAxVp2QIHHef7MlIQFnoECDMQAQ&usg=AOvVaw0oS2sNVkApYvcS_BsC8s3p): A mathematical correlation coefficient representing the relationship between one 2D slice of original vs. corresponding scrambled 2D slice.
- Structural Similarity Index Measure[[SSIM]](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://en.wikipedia.org/wiki/Structural_similarity_index_measure&ved=2ahUKEwjWrMKvktKNAxUR1QIHHcsFMcsQFnoECBAQAQ&usg=AOvVaw0DeKf5zozxPyKboaovBeRN): A perceptual metric that quantifies image quality degradation caused by scrambling.
- Identical[[np.allclose]](https://numpy.org/doc/2.1/reference/generated/numpy.allclose.html): Searches for exact identical arrays of voxels
## What to expect:
- At the bottom of each test, we visualize either plots to track leakage or a report inlcuding basic descriptive statistics with distribution plots. 
- The x-axis in the plots shows scrambling parameter, and y-axis shows leakage score.
  
## List of tests:
- Original Vs. Original
- Permutation
- Permutation - channel replaced

In [3]:
import os, sys, subprocess, csv, random, wget, shutil
import numpy as np
from IPython.display import HTML, display
import matplotlib.pyplot as plt

In [6]:
def viz(f, crit): 
    pl = [float(x[0]) for x in f]
    fl = [int(x[1]) for x in f]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
    ax1.plot(list(crit),pl, color='skyblue')
    ax1.set_title("Partial Leakage")
    ax1.set_xlabel("Scrambling parameter")
    ax1.set_ylabel("Leakage score")
    ax2.plot(list(crit),fl, color='skyblue')
    ax2.set_title("Full Leakage")
    ax2.set_xlabel("Scrambling parameter")
    
    plt.tight_layout()
    plt.show()

def run_test(files=list, opt=list, scramble=True, clean=True, disp=False):
    
    os.makedirs("img", exist_ok=True)
    link_original = "https://s3.amazonaws.com/openneuro.org/ds000117/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_meg.fif?versionId=RYisEgBpM3WNBoVpb2qE2EXs4oNJFn3x"
    wget.download(link_original)
    os.makedirs("input/original", exist_ok=True)
    file_ = [os.path.join(root, file) for root, _, files in os.walk(os.getcwd()) for file in files if file.endswith(".fif")]
    shutil.copy2(file_[0], "input/original")
    os.remove(file_[0])
    print(f"\nOriginal file downloaded.")
    script = os.path.join(os.path.dirname(os.getcwd()), "run_test.py")
    original = files[0]
    scrambled = files[1]
    params = {
        "input": original,
        "output": scrambled,
        "opts": opts
    }
    if scramble:
        ! scramble "{params['input']}" "{params['output']}" {" ".join(params['opts'])}
    print(f"Original file is scrambled.")
    result= subprocess.run(
        ["python3", script, original, scrambled, "True"],
        capture_output=True,
        text=True
    )
    print(f"Analysis is finished.")
    print(result.stdout)

    if disp:
        file_ = [os.path.join(root, file) for root, _, files in os.walk(os.getcwd()+"/report/") for file in files if file.endswith(".html")]
        with open(file_[0]) as f:
            display(HTML(f.read()))
  
    print(f"Cleaning up ...")    
    if clean:
        ! rm -r input report img


# MEG

# Method: Original Vs. Original
We test same original image and we expect to see completely partial leakage as well as full leakage.
In the visualization you can see the distribution of **np.allclose** which identifies all slices are similar.

In [9]:
original = "input/original/"
scrambled = "input/scrambled/"
files = [original, original]
run_test(files, scramble=False, clean=False, disp=True) # NOTE: If original vs. original, then clean=False, otherwise original file gets deleted.

100% [..................................................] 860240688 / 860240688
Original file downloaded.
Original file is scrambled.
Analysis is finished.
########################################
 - Subject ID: sub-01
 - Task: facerecognition
 - Run: run-01
 - Shape: (395, 540100)
########################################
TASK! viz not implemented!
	 - Temporal: 	Full Leakage: 393/395 channels 	Partial Leakage 1.0

Total time taken: 00:00:14



IndexError: list index out of range

# Method: Permutation
In this case we **permute** the image to see how scramble affect the leakage.
Permutation is done in two different ways.

- Independently where we pass **-i** as argument, which all arrays in the given axis are permuted independently.
- Without **-i** the given axis remain as it is, while other axis are permuted.
  
We test permutation on each axis w/o passing **-i** as argument

# EEG

# Original Vs. Original
We test the same original image and we expect to see full leakage.

For more details, we can check the visualization where you can see the distribution of identicals.
## What to expect:
Since we're passing the same image and consider it as scramble, we expect to see identified full leakage, and the distribution plots in each axis should show **n** number of slices that is identical. **n** is equal to the number of images each axis contains.